<div style="display:flex;flex-direction:row;justify-content: space-evenly;">
<img src="tecnm.png" style="height:100px;"/>
<img src="itcolima.svg" style="width:100px"/>
</div>
<div style="display:flex;flex-direction:column;gap:20px;text-align:center">
<h1>Tecnológico Nacional de México campus Colima</h1>
<h2>Maestría en Sistemas Computacionales</h2>
<h2>Tecnologías de programación</h2>
<h2>Actividad - "Primitivas de sincronización"</h2>
<h2>D. en C. Patricia Elizabeth Figueroa Millán</h2>
<h3>Dante Alberto Márquez Almaraz | G2146002</h3>
<p style="text-align:right;">Villa de Álvarez, Colima - 08 de noviembre de 2022</p>
<p></p>
</div>

Objetivo:

Investigar las Primitivas de Sincronización de Hilos que ofrece Python:

Lock

Event

Condition

Semaphore

BoundedSemaphore

Barrier

Metodología:

Se realizó una investigación referente a los temas de sincronización y se complementaron con ejemplos de código en Python.

Materiales:

Computadora.
Visual Studio Code.
Jupyter Notebook.


Desarrollo:

 Las primitivas de sincronización proporcionan un mecanismo diferente para la señalización: al liberar un bloqueo, un subproceso notifica a otro subproceso que puede continuar adquiriendo el bloqueo [1].



Resultados:

LOCK

Un Lock se puede usar para proteger el acceso a un recurso compartido. Solamente el titular del bloqueo puede utilizar el recurso. Múltiples intentos de adquirir la cerradura se bloquearán de modo que solo haya un titular en cada momento [2].

In [15]:
import threading
Lock=threading.Lock()
ITERS = 1000000
x = [0]

def worker():
    for _ in range(ITERS):
        Lock.acquire()
        x[0] += 1
        Lock.release()


        #with Lock:
        #    x[0] += 1
def main():
    x[0] = 0  # you may use `global x` instead of this list trick too
    t1 = threading.Thread(target=worker)
    t2 = threading.Thread(target=worker)

    t1.start()
    t2.start()
    t1.join()
    t2.join()

for i in range(5):
    main()
    print(f'iteration {i}. expected x = {ITERS*2}, got {x[0]}')

iteration 0. expected x = 2000000, got 2000000
iteration 1. expected x = 2000000, got 2000000
iteration 2. expected x = 2000000, got 2000000
iteration 3. expected x = 2000000, got 2000000
iteration 4. expected x = 2000000, got 2000000


EVENT

Los eventos son objetos que se utilizan para la comunicación entre hilos. Un subproceso espera una señal mientras otro la emite. Básicamente, un objeto de evento administra un indicador interno que se puede establecer en verdadero con el método set() y restablecer en falso con el método clear(). El método wait() bloquea hasta que la bandera es verdadera [3].

In [16]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,format='(%(threadName)-9s) %(message)s',)
                    
def wait_for_event(e):
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    while not e.isSet():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other things')

if __name__ == '__main__':
    e = threading.Event()
    t1 = threading.Thread(name='blocking', target=wait_for_event,args=(e,))
    t1.start()

    t2 = threading.Thread(name='non-blocking', target=wait_for_event_timeout, args=(e, 2))
    t2.start()

    logging.debug('Waiting before calling Event.set()')
    time.sleep(3)
    e.set()
    logging.debug('Event is set')

(blocking ) wait_for_event starting
C:\Users\dante\AppData\Local\Temp\ipykernel_16348\3569324192.py:13: DeprecationWarning: isSet() is deprecated, use is_set() instead
  while not e.isSet():
(non-blocking) wait_for_event_timeout starting
(MainThread) Waiting before calling Event.set()
(non-blocking) event set: False
(non-blocking) doing other things
(non-blocking) wait_for_event_timeout starting
(MainThread) Event is set
(blocking ) event set: True
(non-blocking) event set: True
(non-blocking) processing event


CONDITION

Debido a que una variable de condición siempre está asociada con algún tipo de bloqueo, puede vincularse a un recurso compartido. Se puede pasar un bloqueo o se creará uno de forma predeterminada. Pasar uno es útil cuando varias variables de condición deben compartir el mismo bloqueo. El bloqueo es parte del objeto de condición: no tenemos que rastrearlo por separado. Por lo tanto, el objeto de condición permite que los subprocesos esperen a que se actualice el recurso [4].

In [17]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,format='(%(threadName)-9s) %(message)s',)

def consumer(cv):
    logging.debug('Consumer thread started ...')
    with cv:
        logging.debug('Consumer waiting ...')
    cv.wait()
    logging.debug('Consumer consumed the resource')

def producer(cv):
    logging.debug('Producer thread started ...')
    with cv:
        logging.debug('Making resource available')
        logging.debug('Notifying to all consumers')
        cv.notifyAll()

if __name__ == '__main__':
    condition = threading.Condition()
    cs1 = threading.Thread(name='consumer1', target=consumer, args=(condition,))
    cs2 = threading.Thread(name='consumer2', target=consumer, args=(condition,))
    pd = threading.Thread(name='producer', target=producer, args=(condition,))

    cs1.start()
    time.sleep(2)
    cs2.start()
    time.sleep(2)
    pd.start()


(consumer1) Consumer thread started ...
(consumer1) Consumer waiting ...
Exception in thread consumer1:
Traceback (most recent call last):
  File "c:\Users\dante\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\dante\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dante\AppData\Local\Temp\ipykernel_16348\3197485847.py", line 11, in consumer
  File "c:\Users\dante\AppData\Local\Programs\Python\Python310\lib\threading.py", line 312, in wait
    raise RuntimeError("cannot wait on un-acquired lock")
RuntimeError: cannot wait on un-acquired lock
(consumer2) Consumer thread started ...
(consumer2) Consumer waiting ...
Exception in thread consumer2:
Traceback (most recent call last):
  File "c:\Users\dante\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\dante

SEMAPHORE

Semaphore es simplemente una variable que no es negativa y se comparte entre subprocesos. Mientras que los objetos Lock y RLock permiten que solo se ejecute un subproceso, Semaphore permite que se ejecute más de un subproceso a la vez. Los semáforos se utilizan para proteger los recursos que tienen una capacidad limitada al especificar la cantidad de subprocesos que se pueden ejecutar al crear un objeto Semaphore. Si este conteo inicial es 1, Semaphores puede ayudar en la sincronización de hilos [5].

In [18]:
import threading
import time

read_mutex = threading.Semaphore(3)

# Our shared resource
data = "A Data Stream"

class ReaderThread(threading.Thread):
  def __init__(self):
    threading.Thread.__init__(self)

  def run(self):

    read_mutex.acquire()

    output = self.getName() + " starts reading"
    print(output)
  
    # threads take time to read a data
    time.sleep(0.5)
    some_data = data

    output = self.getName() + " ends reading"
    print(output)

    read_mutex.release()

if __name__ == "__main__":

  threads = []
  for i in range(10):
    threads.append(ReaderThread())

  for thread in threads:
    thread.start()

  for thread in threads:
    thread.join()

C:\Users\dante\AppData\Local\Temp\ipykernel_16348\282423170.py:17: DeprecationWarning: getName() is deprecated, get the name attribute instead
  output = self.getName() + " starts reading"


Thread-83 starts reading
Thread-84 starts reading
Thread-85 starts reading


C:\Users\dante\AppData\Local\Temp\ipykernel_16348\282423170.py:24: DeprecationWarning: getName() is deprecated, get the name attribute instead
  output = self.getName() + " ends reading"


Thread-84 ends reading
Thread-85 ends reading
Thread-83 ends reading
Thread-88 starts reading
Thread-86 starts reading
Thread-87 starts reading
Thread-87 ends reading
Thread-86 ends reading
Thread-88 ends reading
Thread-91 starts reading
Thread-90 starts reading
Thread-89 starts reading
Thread-91 ends readingThread-89 ends reading
Thread-90 ends reading
Thread-92 starts reading

Thread-92 ends reading


BOUNDEDSEMAPHORE

Se puede usar una clase threading.BoundedSemaphore que verificará que el contador interno no supere el valor inicial a través de llamadas a release(). Si lo hace, se generará un ValueError, tratándolo como una condición de error [6].

In [19]:
from threading import Semaphore, BoundedSemaphore

# Usually, you create a Semaphore that will allow a certain number of threads
# into a section of code. This one starts at 5.
s1 = Semaphore(5)

# When you want to enter the section of code, you acquire it first.
# That lowers it to 4. (Four more threads could enter this section.)
s1.acquire()

# Then you do whatever sensitive thing needed to be restricted to five threads.

# When you're finished, you release the semaphore, and it goes back to 5.
s1.release()


# That's all fine, but you can also release it without acquiring it first.
s1.release()

# The counter is now 6! That might make sense in some situations, but not in most.
print(s1._value)  # => 6

# If that doesn't make sense in your situation, use a BoundedSemaphore.

s2 = BoundedSemaphore(5)  # Start at 5.

s2.acquire()  # Lower to 4.

s2.release()  # Go back to 5.

try:
    s2.release()  # Try to raise to 6, above starting value.
except ValueError:
    print('As expected, it complained.') 

6
As expected, it complained.


BARRIER

Permite que varios subprocesos esperen en la misma instancia de objeto de barrera (por ejemplo, en el mismo punto del código) hasta que llegue un número fijo predefinido de subprocesos (por ejemplo, la barrera está llena), después de lo cual todos los subprocesos son notificados y liberados para continuar con su ejecución. .

Internamente, una barrera mantiene un recuento de la cantidad de subprocesos que esperan en la barrera y una cantidad máxima configurada de partes (subprocesos) que se esperan. Una vez que el número esperado de partes alcanza el máximo predefinido, se notifica a todos los subprocesos en espera [7].

In [20]:
# SuperFastPython.com
# example of using a barrier
from time import sleep
from random import random
from threading import Thread
from threading import Barrier

# target function to prepare some work
def task(barrier, number):
    # generate a unique value
    value = random() * 10
    # block for a moment
    sleep(value)
    # report result
    print(f'Thread {number} done, got: {value}')
    # wait on all other threads to complete
    barrier.wait()

# create a barrier
barrier = Barrier(5 + 1)
# create the worker threads
for i in range(5):
    # start a new thread to perform some work
    worker = Thread(target=task, args=(barrier, i))
    worker.start()
# wait for all threads to finish
print('Main thread waiting on all results...')
barrier.wait()
# report once all threads are done
print('All threads have their result')

Main thread waiting on all results...
Thread 2 done, got: 1.603247531739026
Thread 3 done, got: 3.7205210257872725
Thread 0 done, got: 4.183958044193821
Thread 1 done, got: 4.342350786212955
Thread 4 done, got: 9.142478677033614
All threads have their result


Conclusiones:



En este trabajo, hemos aprendido la sincronización en Python para evitar condiciones de carrera mediante el uso del módulo de subprocesamiento en Python para lograr la sincronización.

Referencias:

[1] https://learn.microsoft.com/es-es/dotnet/standard/threading/overview-of-synchronization-primitives

[2] https://rico-schmidt.name/pymotw-3/asyncio/synchronization.html

[3] https://subscription.packtpub.com/book/application-development/9781785289583/2/ch02lvl1sec29/thread-synchronization-with-an-event

[4] https://www.bogotobogo.com/python/Multithread/python_multithreading_Synchronization_Condition_Objects_Producer_Consumer.php

[5] https://www.askpython.com/python/examples/synchronization-in-python

[6] https://stackoverflow.com/questions/48971121/what-is-the-difference-between-semaphore-and-boundedsemaphore

[7] https://superfastpython.com/thread-barrier-in-python/

[8] https://docs.python.org/es/3/library/asyncio-sync.html

